In [5]:
import pandas as pd
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from imblearn.over_sampling import RandomOverSampler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

data_file = "dat.csv"

df = pd.read_csv(data_file)

cols = ["day_of_week", "hour_of_day", "motion_duration"]
X = df[cols].values
y = df["label"].values

print(f"dataset balance: {Counter(y)}")

sampler = RandomOverSampler(sampling_strategy="minority")

X, y = sampler.fit_resample(X, y)

print(f"dataset balance: {Counter(y)}")

dataset balance: Counter({np.int64(0): 522, np.int64(1): 186})
dataset balance: Counter({np.int64(1): 522, np.int64(0): 522})


In [6]:
scaler = MinMaxScaler()

df[cols] = scaler.fit_transform(df[cols]) # to normalize values since some are orders of magnitude greater than others, e.g. motion_duration

print(df)

SEQ_LEN = 10

X_seq, Y_seq = [], []

# sliding window to make preds on each timestep
for i in range(len(X) - SEQ_LEN):
    X_seq.append(X[i:i+SEQ_LEN])
    Y_seq.append(y[i+SEQ_LEN])

X_train, X_test, Y_train, Y_test = train_test_split(X_seq, Y_seq, test_size=0.2, random_state=32)

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32)

print(f"X_train shape: {X_train.shape}")
print(f"Y_train shape: {Y_train.shape}")


train_loader = DataLoader(TensorDataset(X_train, Y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, Y_test), batch_size=32)

               timestamp  day_of_week  hour_of_day  motion_duration  label
0    2024-05-20 02:15:00          0.0     0.086957         0.749948      1
1    2024-05-20 07:15:00          0.0     0.304348         0.017295      0
2    2024-05-20 12:30:00          0.0     0.521739         0.033132      0
3    2024-05-20 14:00:00          0.0     0.608696         0.001875      0
4    2024-05-20 18:45:00          0.0     0.782609         0.062305      0
..                   ...          ...          ...              ...    ...
703  2024-09-15 03:15:00          1.0     0.130435         0.266514      0
704  2024-09-15 07:35:00          1.0     0.304348         0.070640      1
705  2024-09-15 11:55:00          1.0     0.478261         0.053136      1
706  2024-09-15 18:35:00          1.0     0.782609         0.074807      0
707  2024-09-15 23:00:00          1.0     1.000000         0.062305      0

[708 rows x 5 columns]
X_train shape: torch.Size([827, 10, 3])
Y_train shape: torch.Size([827])


In [7]:
class Detector(nn.Module):
    def __init__(self, input_size=3, hidden_size=32, num_layers=2, bidirectional=True):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, bidirectional=bidirectional) # perhaps make bidirectional to learn patterns relating to other motions
        lstm_output_size = hidden_size * 2 if bidirectional else hidden_size
        self.fc = nn.Linear(lstm_output_size, 1)
        print(f"creating Detector with hidden_size={hidden_size}, and num_layers={num_layers}")

    def forward(self, x):
        out, _ = self.lstm(x)
        return torch.sigmoid(self.fc(out[:, -1, :])).squeeze()


In [8]:
model = Detector()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

creating Detector with hidden_size=32, and num_layers=2


In [9]:
for epoch in range(100):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)  # shape: [batch_size]
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, Y_test)
        print(f"Test Loss: {test_loss.item():.4f}")

Epoch 1, Loss: 0.6334
Test Loss: 0.5427
Epoch 2, Loss: 0.4352
Test Loss: 0.4356
Epoch 3, Loss: 0.3575
Test Loss: 0.3876
Epoch 4, Loss: 0.3172
Test Loss: 0.3523
Epoch 5, Loss: 0.2790
Test Loss: 0.3200
Epoch 6, Loss: 0.2504
Test Loss: 0.2915
Epoch 7, Loss: 0.2225
Test Loss: 0.2640
Epoch 8, Loss: 0.1929
Test Loss: 0.2341
Epoch 9, Loss: 0.1652
Test Loss: 0.2158
Epoch 10, Loss: 0.1470
Test Loss: 0.1951
Epoch 11, Loss: 0.1323
Test Loss: 0.1768
Epoch 12, Loss: 0.1151
Test Loss: 0.1951
Epoch 13, Loss: 0.1002
Test Loss: 0.1520
Epoch 14, Loss: 0.0832
Test Loss: 0.1254
Epoch 15, Loss: 0.0766
Test Loss: 0.1386
Epoch 16, Loss: 0.0716
Test Loss: 0.1049
Epoch 17, Loss: 0.0536
Test Loss: 0.1142
Epoch 18, Loss: 0.0486
Test Loss: 0.0837
Epoch 19, Loss: 0.0415
Test Loss: 0.0763
Epoch 20, Loss: 0.0371
Test Loss: 0.0724
Epoch 21, Loss: 0.0343
Test Loss: 0.0689
Epoch 22, Loss: 0.0286
Test Loss: 0.0733
Epoch 23, Loss: 0.0268
Test Loss: 0.0522
Epoch 24, Loss: 0.0242
Test Loss: 0.0534
Epoch 25, Loss: 0.0274
Te